In [1]:
# first level direction: alwasy left
# generate clock wise
def xy2d(n, x, y):
    d = 0
    s = int(n / 2)
    while s > 0:
        rx = (x & s) > 0
        ry = (y & s) > 0
        if rx == 0 and ry == 0:
            d += 0
        if rx == 1 and ry == 0:
            d += s*s
        if rx == 1 and ry == 1:
            d += s*s*2
        if rx == 0 and ry == 1:
            d += s*s*3   
#         d += s * s * ((3 * rx) ^ ry)
        x,y = rot(s, x, y, rx, ry)
        s = int(s/2)
    return d

# def d2xy(n, d):
#     t = d
#     x = 0
#     y = 0
#     s = 1
#     while s < n:
#         rx = 1 & int(t/2)
#         ry = 1 & (t^rx)
#         x,y = rot(s, x, y, rx, ry)
#         x += s * rx
#         y += s * ry
#         t = int(t/4)
#         s *= 2
#     return x, y

def rot(n, x, y, rx, ry):
    if rx == 0 : # the upper level direction remains the same, so only change the lower level
        if ry == 1 :
            x = n-1 - x
            y = n-1 - y
        t  = x
        x = y
        y = t
    return x, y

# ===== Testing =====
# for i in range(8):
#     for j in range(8):
#         print(xy2d(8, i, j))

In [2]:
from enum import Enum
class Direction(Enum):
    UP = 0
    LEFT = 1
    RIGHT = 2
    DOWN = 3

class ClockDirection(Enum):
    CLOCKWISE = 0
    COUNTERCLOCKWISE = 1
    UNAVAILABLE = 2

class Quadrant(Enum):
    LOWERLEFT = 0
    LOWERRIGHT = 1
    UPPERRIGHT = 2
    UPPERLEFT = 3
    
next_level_direction=[[Direction.UP, Direction.UP, Direction.RIGHT, Direction.LEFT],
                      [Direction.DOWN, Direction.LEFT, Direction.LEFT, Direction.UP],
                      [Direction.RIGHT, Direction.DOWN, Direction.UP, Direction.RIGHT],
                      [Direction.LEFT, Direction.RIGHT, Direction.DOWN, Direction.DOWN]]

next_level_clock_direction = [[ClockDirection.CLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.UNAVAILABLE],
                           [ClockDirection.CLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.UNAVAILABLE,ClockDirection.CLOCKWISE],
                           [ClockDirection.CLOCKWISE,ClockDirection.UNAVAILABLE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.CLOCKWISE],
                           [ClockDirection.UNAVAILABLE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.CLOCKWISE]]

# ===== Testing =====
# for i in range(4):
#     for j in range(4):
#         print(next_level_direction[i][j])
vard = Direction.UP
# if vard.name == "UP":
#     print("yes") # yes
# print(vard.name)
# print(vard.value)
# if vard == Direction.UP:
#     print("yes")
# for dir in Direction:
#     print(dir)

In [3]:
# dictionary for interval:
# lower_x
# upper_X
# lower_y
# upper_y
# lower_hilbert
# upper_hilbert


def range_decompose(max_bits, current_bits, down_x, up_x, down_y, up_y, center_x, center_y, pre_dir, this_dir, intervals):
    
    # if there is only 1 element here
    if current_bits == 0:
        hvalue = xy2d(1<<max_bits, down_x, down_y)
        interval = {"lower_x":down_x, "upper_x":up_x, "lower_y":down_y,"upper_y":up_y,"lower_hilbert":hvalue,"upper_hilbert":hvalue}
        intervals.append(interval);
        return
    
    half_region = 1 << (current_bits - 1)
    clock_dir = next_level_clock_direction[pre_dir.value][this_dir.value]
    if current_bits == max_bits:
        clocl_dir = ClockDirection.COUNTERCLOCKWISE
        
    # check if the range contains the entire region
    if down_x == center_x - half_region and down_y == center_y - half_region and up_x == center_x + half_region-1 and up_y == center_y + half_region-1:
        # if yes, return this region as an interval
        lower_x = 0
        upper_x = 0
        lower_y = 0
        upper_y = 0
        if clock_dir.name == "CLOCKWISE":
            if this_dir.name == "UP":
                lower_x = up_x
                upper_x = down_x
                lower_y = up_y
                upper_y = up_y
            elif this_dir.name == "LEFT":
                lower_x = down_x
                upper_x = down_x
                lower_y = up_y
                upper_y = down_y
            elif this_dir.name == "RIGHT":
                lower_x = up_x
                upper_x = up_x
                lower_y = down_y
                upper_y = up_y
            elif this_dir.name == "DOWN":
                lower_x = down_x
                upper_x = up_x
                lower_y = down_y
                upper_y = down_y
        elif clock_dir.name == "COUNTERCLOCKWISE":
            if this_dir.name == "UP":
                lower_x = down_x
                upper_x = up_x
                lower_y = up_y
                upper_y = up_y
            elif this_dir.name == "LEFT":
                lower_x = down_x
                upper_x = down_x
                lower_y = down_y
                upper_y = up_y
            elif this_dir.name == "RIGHT":
                lower_x = up_x
                upper_x = up_x
                lower_y = up_y
                upper_y = down_y
            elif this_dir.name == "DOWN":
                lower_x = up_x
                upper_x = down_x
                lower_y = down_y
                upper_y = down_y
                
        lowhvalue = xy2d(1<<max_bits,lower_x,lower_y)
        uphvalue = xy2d(1<<max_bits,upper_x,upper_y)
        interval = {"lower_x":lower_x, "upper_x":upper_x, "lower_y":lower_y,"upper_y":upper_y,"lower_hilbert":lowhvalue,"upper_hilbert":uphvalue}
        intervals.append(interval)
    else:
        # if not, further decompose it
        current_bits -= 1
        if current_bits <= 0:
            half_region = 0
        else:
            half_region = 1 << (current_bits - 1)
        
        if clock_dir.name == "CLOCKWISE":
            if this_dir.name == "UP":
                parent_dir = Direction.UP
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "LEFT":
                parent_dir = Direction.LEFT
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "RIGHT":
                parent_dir = Direction.RIGHT
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "DOWN":
                parent_dir = Direction.DOWN
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
        elif clock_dir.name == "COUNTERCLOCKWISE":
            if this_dir.name == "UP":
                parent_dir = Direction.UP
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "LEFT":
                parent_dir = Direction.LEFT
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "RIGHT":
                parent_dir = Direction.RIGHT
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "DOWN":
                parent_dir = Direction.DOWN
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)

# ===== Testing =====
# intervals = []
# range_decompose(3,3,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,0,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,1,2,3,2,3,3,3,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,1,2,3,2,3,3,3,Direction.DOWN,Direction.DOWN,intervals)
# print(intervals)

# import ipdb
# ipdb.set_trace()
# intervals = []
# range_decompose(3,3,0,3,0,7,4,4,Direction.LEFT,Direction.LEFT,intervals)
# print(len(intervals))
# intervals = intervals[0]
# print(intervals[0])
# print(intervals[0]["lower_x"])
# for inter in intervals:
#     print(intervals)

In [4]:
def merge_intervals(intervals):
    if len(intervals) <= 1:
        return intervals
    merged_intervals = []
    i = 0
    j = 0
    while i < len(intervals):
        temp_upper = intervals[i]["upper_hilbert"]
        j = 1
        while i+j < len(intervals) and intervals[i+j]["lower_hilbert"] == temp_upper + 1:
            temp_upper = intervals[i+j]["upper_hilbert"]
            j += 1
        interval = {"lower_x":intervals[i]["lower_x"], "upper_x":intervals[i+j-1]["upper_x"], 
                    "lower_y":intervals[i]["lower_y"],"upper_y":intervals[i+j-1]["upper_y"],
                    "lower_hilbert":intervals[i]["lower_hilbert"],"upper_hilbert":intervals[i+j-1]["upper_hilbert"]}
        merged_intervals.append(interval)
        i += j
    return merged_intervals

def get_intervals(lower_x, upper_x, lower_y, upper_y, max_bits = 22):
    intervals = []
    range_decompose(max_bits,max_bits,lower_x,upper_x,lower_y,upper_y,1<<(max_bits-1),1<<(max_bits-1),Direction.LEFT,Direction.LEFT,intervals)
    intervals = merge_intervals(intervals)
    return intervals
    
# ===== Testing =====
#print(get_intervals(2,4,2,4))

In [5]:
# ===== Testing =====
# import ipdb
# ipdb.set_trace()
# intervals = []
# range_decompose(3,3,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# print(intervals)
# intervals = merge_intervals(intervals)
# print(intervals)

In [10]:
def range_query(lower_x, upper_x, lower_y, upper_y,max_bits = 22):
    lx = int((2*lower_x+180)*10000) >> 22-max_bits
    ly = int((lower_y+180)*10000) >> 22-max_bits
    ux = int((2*upper_x+180)*10000) >> 22-max_bits
    uy = int((upper_y+180)*10000) >> 22-max_bits
    
    intervals = get_intervals(lx,ux,ly,uy,max_bits)
    print("total intervals: ",len(intervals))
#     print(intervals)
    total_size = len(intervals)
#     cost = 0
#     for interval in intervals:
        
        

In [12]:
# import ipdb
# ipdb.set_trace()
range_query(1, 2, 1, 2,14)
range_query(1, 10, 1, 10,14)
range_query(-10, 10, -10, 10,14)
range_query(-20, 20, -20, 20,14)
range_query(-30, 30, -30, 30,14)
range_query(-40, 40, -40, 40,14)
range_query(-15.1234,17.1983,-13.1677,11.8765,14)

# range_query(38.7900,39.0118,-77.2123,-76.8835,14) # Washington DC
# range_query(22.1481,22.5457,113.8436,114.4379,14) # Hong Kong
# range_query(22.2201,22.2841,114.1278,114.2572,14) # Hong Kong island

total intervals:  52
total intervals:  534
total intervals:  794
total intervals:  2218
total intervals:  3041
total intervals:  3474
total intervals:  1109


In [ ]:
import csv
HILBERT_N = 4194306 # 4 Million
list = []
count = -1
with open("./Data/SortedPOIs.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if count == -1: # ignore the head info
            count += 1
            continue
        x = float(row[0])
        y = float(row[1])
        x_ = int((2*x+180)*10000)
        y_ = int((y+180)*10000)
        h_ = xy2d(HILBERT_N,x_, y_)
        list.append([x,y,x_,y_,h_,count])
        count += 1
        if count % 10000 == 0:
            print(count)

In [ ]:
import csv
def write_csv_file(path, head, data):  
    try:  
        with open(path, 'w', newline='') as csv_file:  
            writer = csv.writer(csv_file, dialect='excel')  
  
            if head is not None:  
                writer.writerow(head)  
  
            for row in data:  
                writer.writerow(row)  
  
            print("Write a CSV file to path %s Successful." % path)  
    except Exception as e:  
        print("Write an CSV file to path: %s, Case: %s" % (path, e)) 

In [ ]:
list.sort(key = lambda x:(x[4]))
for i in  range(len(list)):
    list[i][5] = i
write_csv_file("./Data/HilbertSortedPOIs.csv",['x','y','x_','y_','hilbert','order','prediction'],list)